In [1]:
from qiskit import ClassicalRegister, QuantumRegister
from qiskit import QuantumCircuit,compile, execute,IBMQ
from qiskit import BasicAer
from qiskit.tools.visualization import circuit_drawer
from qiskit.tools.visualization import plot_histogram
from qiskit.providers.ibmq import least_busy
from math import pi
import matplotlib.pyplot as plt
import sys
from qiskit.tools.visualization import circuit_drawer
from parseRealization import *
%matplotlib inline
testDir="./tests/"

C:\Users\krylo\Anaconda3\lib\site-packages\marshmallow\schema.py:364: ChangedInMarshmallow3Warning: strict=False is not recommended. In marshmallow 3.0, schemas will always be strict. See https://marshmallow.readthedocs.io/en/latest/upgrading.html#schemas-are-always-strict
  ChangedInMarshmallow3Warning


In [2]:
#select a backend
IBMQ.load_accounts()
backends = IBMQ.backends(simulator=False,filters=lambda x: x.configuration().n_qubits > 4)
least_busy =least_busy(backends)
#print(available)
#least_busy = BasicAer.get_backend('qasm_simulator')
#least_busy = IBMQ.get_backend('ibmq_16_melbourne')
couplingMap = least_busy.configuration().coupling_map
print(couplingMap)
qubitsSize = least_busy.configuration().n_qubits


[[1, 0], [2, 0], [2, 1], [3, 2], [3, 4], [4, 2]]


In [3]:
def bigFunction(fileName):
    ioClass = readCircuitInformation(fileName) 
    answers = ioClass.getKmap()
    size = ioClass.getSize()
    ioClass.readGatesFromFile(fileName)
    backend_sim = BasicAer.get_backend('qasm_simulator')
    error_count = 0
    ctg = CircuitTransitionGraph()
    print(ctg.transformCoupling(couplingMap))
    ctg.setSize(size)
    for i in range(0,2):
        #This part corresponds for part of experiment without minimal or no changes
        qr,cr,qc = ioClass.createCircuitAndSetInput(i)
        qc,qr = ctg.readGatesFromIOClass(qr,qc, ioClass)
        measureFidelityWithoutChanges(qr,cr,qc)
        
        #This part corresponds for part of experiment after significant changes
        qr,cr,qc = ioClass.createCircuitAndSetInput(i)
        ctg = fixTheStuff(ctg)
        #This one needs to comply with changes you did
        qc,qr = ctg.readFixedGatesFromCtg(qr,qc)
        measureFidelityWithChanges(qr,cr,qc)
    
def fixTheStuff(ctg):
    print("Missing connections are",ctg.getMissingConnections())
    #This one to fix the changes... fixthemissingedges connects stuff around. did not test though
    ctg.fixMissingEdges()
    print("FIxing the stuff")
    return ctg


def measureFidelityWithoutChanges(qr,cr,qc):
    least_busy = BasicAer.get_backend('qasm_simulator')
    qc.measure(qr,cr)
    job = execute(qc,least_busy,shots=200)
    result = job.result()
    print(result.get_counts())
    #plot_histogram(result.get_counts())

    
#This needs to be implemented    
def measureFidelityWithChanges(qr,cr,qc):
    least_busy = BasicAer.get_backend('qasm_simulator')
    qc.measure(qr,cr)
    job = execute(qc,least_busy,shots=200)
    result = job.result()
    print(result.get_counts())
    #plot_histogram(result.get_counts())


In [4]:
fileName="ham3_28"
#if testFromFile(filename) == 0:
fileName = testDir+fileName
bigFunction(fileName)

./tests/ham3_28.pla
The constants are set to .constants ---

The circuit has no garbage lines in it or file parsed incorrectly, check ioclass
The circuit has no constant inputs lines in it or file parsed incorrectly, check ioclass
{'b': {'a', 'c'}, 'a': {'b', 'c'}, 'c': {'b', 'e', 'a', 'd'}, 'd': {'e', 'c'}, 'e': {'c', 'd'}}
{'000': 200}
Self skeleton is ['ab', 'bc', 'ab', 'bc', 'bc', 'ab', 'bc', 'bc', 'ab', 'ab', 'bc', 'ab', 'bc', 'bc', 'ac', 'bc']
Missing connections are []
b
a
c
d
e
FIxing the stuff
{'000': 200}
{'010': 200}
Self skeleton is ['ab', 'bc', 'ab', 'bc', 'bc', 'ab', 'bc', 'bc', 'ab', 'ab', 'bc', 'ab', 'bc', 'bc', 'ac', 'bc']
Missing connections are []
b
a
c
d
e
FIxing the stuff
{'010': 200}


C:\Users\krylo\Anaconda3\lib\site-packages\marshmallow\schema.py:364: ChangedInMarshmallow3Warning: strict=False is not recommended. In marshmallow 3.0, schemas will always be strict. See https://marshmallow.readthedocs.io/en/latest/upgrading.html#schemas-are-always-strict
  ChangedInMarshmallow3Warning
